In [ ]:
import pandas as pd
import tsfresh

In [63]:
X = pd.read_hdf('data/achat_filtered.h5')
y = pd.read_hdf('data/achat_y.h5')
y.reset_index(inplace=True, drop=True)
X_filt = tsfresh.feature_selection.select_features(X,y, ml_task='classification')

In [60]:
good_sigs = [42,3,56,47,7,2,57,52]
good_features = []
for col in good_sigs:
    good_features += list(X_filt.columns[X_filt.columns.str.startswith(str(col))])

In [61]:
X_good = X_filt[good_features]

In [55]:
subject = 0
for i in range(1,6):
    X.loc[(i-1)*90:i*90, 'subject'] = subject
    subject += 1

In [59]:
X_0 = pd.read_hdf('data/features/filtered/fixed/filt_achat_0_eff.h5')

In [47]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import numpy as np
rf = RandomForestClassifier(n_estimators = 200, max_depth=3, random_state=0)
print('\n RANDOM FOREST \n Mean 5-fold cross-validation score = '+str(np.mean(cross_val_score(rf, X_filt,iloc[:,:10], y, cv=5))))


 RANDOM FOREST 
 Mean 3-fold cross-validation score = 0.49782608695652175


In [ ]:
import seaborn as sns

ax = sns.violinplot()